In [1]:
import sys
sys.path.append("../library")

In [2]:
import math
# import columns.py for csv processing
from columns import read_csv, write_csv
from __future__ import print_function, division
from thinkbayes2 import Suite, Pmf, Mean, MeanVar, EvalPoissonPmf, \
    MakeCdfFromList, MakeNormalPmf, MakeMixture, MakePoissonPmf, MakeExponentialPmf, MakeUniformPmf, \
    PmfProbLess, PmfProbEqual
import thinkplot

# Exercise 1
If buses arrive at a bus stop every 20 minutes, and you arrive at the bus stop at a random time, your wait time until the bus arrives is uniformly distributed from 0 to 20 minutes.

But in reality, there is variability in the time between buses. Suppose you are waiting for a bus, and you know **the historical distribution of time between buses.**
Compute your distribution of wait times.

Hint: Suppose that the time between buses is either 5 or 10 minutes with equal probability. What is the probability that you arrive during one of the 10 minute intervals?

In [3]:
# supposed the historical distribution of time between buses is pmf_zb
def PmfOfWaitTime(pmf_zb: Pmf):
    """
    To estimate waiting time distribution based on distribution of time between buses.
    Arguments:
        1, pmf_zb: the historical distribution of time between buses
    Return:
        distribution of waiting time of buses
    """
    metapmf = Pmf()
    for gap, prob in pmf_zb.Items():
        uniform = MakeUniformPmf(0, gap)
        metapmf.Set(uniform, prob)
    pmf_y = MakeMixture(metapmf)
    return pmf_y

# Exercise 2
Suppose that passengers arriving at the bus stop are well-modeled by a **Poisson process with parameter λ**. If you arrive at the stop and find 3 people waiting, what is your **posterior distribution for your waiting time since the last bus arrived**?

### Analysis
* Poisson distribution equation
    $$ P(x) = { \lambda^k e^{-\lambda} \over k!} $$

In [4]:
def BiasPmf(pmf):
    """
    Compute Bias pmf based on observation of customer, as customer believe that what they saw ~ likelihood of happening
    """
    new_pmf = pmf.Copy()
    for x, p in pmf.Items():
        # actually doing this: p[x] * likelihood(~x)
        new_pmf.Mult(x, x)
    new_pmf.Normalize()
    return new_pmf

In [5]:
class WaitTimeCalculator(object):
    """
    calculator for waiting time.
    """
    def __init__(self, pmf_z):
        # prior of z for timeslot between trains distribution
        self.pmf_z = pmf_z
        self.pmf_zb = BiasPmf(pmf=pmf_z)
        
        self.pmf_y = self.PmfOfWaitTime(self.pmf_zb)
        self.pmf_x = self.pmf_y
        
    def PmfOfWaitTime(pmf_zb: Pmf):
        """
        To estimate waiting time distribution based on distribution of time between buses.
        Arguments:
            1, pmf_zb: the historical distribution of time between buses
        Return:
            distribution of waiting time of buses
        """
        metapmf = Pmf()
        for gap, prob in pmf_zb.Items():
            uniform = MakeUniformPmf(0, gap)
            metapmf.Set(uniform, prob)
        pmf_y = MakeMixture(metapmf)
        return pmf_y

In [9]:
# implementation from https://github.com/AllenDowney/ThinkBayes/blob/master/code/redline.py
# data from 

TypeError: Can't instantiate abstract class Suite with abstract methods Likelihood

# Exercise 3
Suppose that you are an ecologist sampling the insect population in a new environment. You deploy 100 traps in a test area and come back the next day to check on them. You find that 37 traps have been triggered, trapping an insect
inside. Once a trap triggers, it cannot trap another insect until it has been reset.

If you reset the traps and come back in two days, how many traps do you expect to find triggered? Compute a posterior predictive distribution for the number of traps.

# Exercise 4
Suppose you are the manager of an apartment building with 100 light bulbs in common areas. It is your responsibility to replace light bulbs when they break.

On January 1, all 100 bulbs are working. When you inspect them on February 1, you find 3 light bulbs out. If you come back on April 1, how many light bulbs do you expect to find broken?

In the previous exercise, you could reasonably assume that an event is equally likely at any time. For light bulbs, the likelihood of failure depends on the age of the bulb. Specifically, old bulbs have an increasing failure rate due to evaporation of the filament.

This problem is more open-ended than some; you will have to make modeling decisions. You might want to read about the Weibull distribution (http: // en.wikipedia. org/ wiki/ Weibull_ distribution ). Or you might want to look around for information about light bulb survival curves.